In [47]:
import tensorflow as tf
import keras
import keras.backend as K
from keras.layers.core import Activation
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Conv1D
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras_self_attention
from sklearn import preprocessing
from keras.callbacks import TensorBoard
%matplotlib inline

In [48]:
# from keras.utils import plot_model
from tensorflow.keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPooling1D
from keras.layers.merge import concatenate
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import LSTM 
from keras.layers import Attention
from keras.layers import Bidirectional
from keras_self_attention import SeqSelfAttention
from sklearn.metrics import mean_squared_error
from math import sqrt
import pydot_ng
import graphviz

import wandb
from wandb.keras import WandbCallback

np.random.seed(5)

# wandb.init(project="C-Mapss-project", entity="raachelzhu")



## Mount the dataset on a drive or specify path

In [49]:
# In the NASA dataset, there are four data subsets (FD001-4). 
# This function is developed to handle any of the data subsets.

In [50]:
def read_data(filename):

    """function to turn the dataset into dataframe with column headings"""

    train_df = pd.read_csv(
        "dataset/" + filename + "/train_" + filename + ".txt", sep=" ", header=None
    )
    train_df.drop(train_df.columns[[26, 27]], axis=1, inplace=True)
    train_df.columns = [
        "id",
        "cycle",
        "setting1",
        "setting2",
        "setting3",
        "s1",
        "s2",
        "s3",
        "s4",
        "s5",
        "s6",
        "s7",
        "s8",
        "s9",
        "s10",
        "s11",
        "s12",
        "s13",
        "s14",
        "s15",
        "s16",
        "s17",
        "s18",
        "s19",
        "s20",
        "s21",
    ]

    train_df = train_df.sort_values(["id", "cycle"])

    test_df = pd.read_csv(
        "dataset/" + filename + "/test_" + filename + ".txt", sep=" ", header=None
    )
    test_df.drop(test_df.columns[[26, 27]], axis=1, inplace=True)
    test_df.columns = [
        "id",
        "cycle",
        "setting1",
        "setting2",
        "setting3",
        "s1",
        "s2",
        "s3",
        "s4",
        "s5",
        "s6",
        "s7",
        "s8",
        "s9",
        "s10",
        "s11",
        "s12",
        "s13",
        "s14",
        "s15",
        "s16",
        "s17",
        "s18",
        "s19",
        "s20",
        "s21",
    ]

    truth_df = pd.read_csv(
        "dataset/" + filename + "/RUL_" + filename + ".txt", sep=" ", header=None
    )
    truth_df.drop(truth_df.columns[[1]], axis=1, inplace=True)
    return train_df, test_df, truth_df


In [51]:
#Specify the data subset to use for training
filename= "FD001"

In [52]:
# Model can be trained using a set of time windows
# ranging from 20 to 120. Specify the window length

sequence_length = 90

# Not all columns in the dataset is used. Select useful signals in the dataset

sequence_cols = ["s" + str(i) for i in set(range(1, 22))] + [
    "setting1",
    "setting2",
    "setting3",
    "cycle_norm",
]

if filename == "FD001":
    cycle = 130
    sequence_cols = [
        x
        for x in sequence_cols
        # if x not in ["s1", "s5", "s6", "s10", "s16", "s18", "s19", "setting3"]
        if x not in ["s1","s2","s3","s4","s5", "s6","s7","s8","s9","s11","s10","s12","s13","s14","s15","s16", "s18", "s19","s20","s21","setting3","setting2"]
    ]
elif filename == "FD003":
    cycle = 130
    sequence_cols = [
        x
        for x in sequence_cols
        if x not in ["s1", "s5", "s16", "s18", "s19", "setting3"]
    ]
else:
    cycle = 130
    sequence_cols = ["s" + str(i) for i in set(range(1, 22))] + [
        "setting1",
        "setting2",
        "setting3",
        "cycle_norm",
    ]


In [53]:
train_df, test_df, truth_df = read_data(filename)

In [54]:
train_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


In [55]:
test_df

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,521.72,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,522.16,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,521.97,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,521.38,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,522.15,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13091,100,194,0.0049,0.0000,100.0,518.67,643.24,1599.45,1415.79,14.62,...,520.69,2388.00,8213.28,8.4715,0.03,394,2388,100.0,38.65,23.1974
13092,100,195,-0.0011,-0.0001,100.0,518.67,643.22,1595.69,1422.05,14.62,...,521.05,2388.09,8210.85,8.4512,0.03,395,2388,100.0,38.57,23.2771
13093,100,196,-0.0006,-0.0003,100.0,518.67,643.44,1593.15,1406.82,14.62,...,521.18,2388.04,8217.24,8.4569,0.03,395,2388,100.0,38.62,23.2051
13094,100,197,-0.0038,0.0001,100.0,518.67,643.26,1594.99,1419.36,14.62,...,521.33,2388.08,8220.48,8.4711,0.03,395,2388,100.0,38.66,23.2699


In [56]:
#Preprocess the dataset

In [57]:
# These two functions preprocess the training and test dataset respectively

def train_data_prep(train_df):
      """
       A function to preprocess the training dataset for piece-wise RUL
       estimation
       
       Parameter:
         
         training dataframe (FD001-4)
        
       Returns:
         
         train_df, a preprocessed dataframe with column names
        and target variable.

      """
      rul = pd.DataFrame(train_df.groupby("id")["cycle"].max()).reset_index()
      rul.columns = ["id", "max"]
      train_df = train_df.merge(rul, on=["id"], how="left")
      train_df["RUL"] = train_df["max"] - train_df["cycle"]
      train_df.drop("max", axis=1, inplace=True)
      train_df["R_early"] = train_df["RUL"].apply(lambda x: cycle if x >= cycle else x)
      train_df = train_df.drop(["RUL"], axis=1)
      return train_df

def test_data_prep(test_df, truth_df):
    """
    A function to preprocess the test data subset for
     piece-wise RUL prediction
          
        Parameters:

          truth_df: the actual RUL
          test_df: the test data subset

        Returns:

          test_df: a preprocessed test dataset with piecewise RUL.
          """
    rul = pd.DataFrame(test_df.groupby("id")["cycle"].max()).reset_index()
    rul.columns = ["id", "max"]
    truth_df.columns = ["more"]
    truth_df["id"] = truth_df.index + 1
    truth_df["max"] = rul["max"] + truth_df["more"]
    truth_df.drop("more", axis=1, inplace=True)
    # generate RUL for test data
    test_df = test_df.merge(truth_df, on=["id"], how="left")
    test_df["RUL"] = test_df["max"] - test_df["cycle"]
    test_df.drop("max", axis=1, inplace=True)
    test_df["R_early"] = test_df["RUL"].apply(lambda x: cycle if x >= cycle else x)
    test_df = test_df.drop(["RUL"], axis=1)
    return test_df


In [58]:
train_df = train_data_prep(train_df)

In [59]:
train_df

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,R_early
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,130
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,130
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,130
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,130
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,-0.0004,-0.0003,100.0,518.67,643.49,1597.98,1428.63,14.62,...,2388.26,8137.60,8.4956,0.03,397,2388,100.0,38.49,22.9735,4
20627,100,197,-0.0016,-0.0005,100.0,518.67,643.54,1604.50,1433.58,14.62,...,2388.22,8136.50,8.5139,0.03,395,2388,100.0,38.30,23.1594,3
20628,100,198,0.0004,0.0000,100.0,518.67,643.42,1602.46,1428.18,14.62,...,2388.24,8141.05,8.5646,0.03,398,2388,100.0,38.44,22.9333,2
20629,100,199,-0.0011,0.0003,100.0,518.67,643.23,1605.26,1426.53,14.62,...,2388.23,8139.29,8.5389,0.03,395,2388,100.0,38.29,23.0640,1


In [60]:
test_df= test_data_prep(test_df, truth_df)

In [61]:
test_df

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,R_early
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735,130
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916,130
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166,130
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737,130
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130,130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13091,100,194,0.0049,0.0000,100.0,518.67,643.24,1599.45,1415.79,14.62,...,2388.00,8213.28,8.4715,0.03,394,2388,100.0,38.65,23.1974,24
13092,100,195,-0.0011,-0.0001,100.0,518.67,643.22,1595.69,1422.05,14.62,...,2388.09,8210.85,8.4512,0.03,395,2388,100.0,38.57,23.2771,23
13093,100,196,-0.0006,-0.0003,100.0,518.67,643.44,1593.15,1406.82,14.62,...,2388.04,8217.24,8.4569,0.03,395,2388,100.0,38.62,23.2051,22
13094,100,197,-0.0038,0.0001,100.0,518.67,643.26,1594.99,1419.36,14.62,...,2388.08,8220.48,8.4711,0.03,395,2388,100.0,38.66,23.2699,21


In [62]:
#Normalize the dataset 

In [63]:
#This function normalizes the training and test dataset.
def train_test_scaler(train_data, test_data):

    """function to standardize the dataset"""

    train_data["cycle_norm"] = train_data["cycle"]
    cols_normalize = train_data.columns.difference(["id", "cycle", "R_early"])
    min_max_scaler = preprocessing.MinMaxScaler()
    norm_train_df = pd.DataFrame(
        min_max_scaler.fit_transform(train_data[cols_normalize]),
        columns=cols_normalize,
        index=train_data.index,
    )
    join_df = train_data[train_data.columns.difference(cols_normalize)].join(
        norm_train_df
    )
    train_df = join_df.reindex(columns=train_data.columns)

    # MinMax normalization (from 0 to 1)
    test_data["cycle_norm"] = test_data["cycle"]
    norm_test_df = pd.DataFrame(
        min_max_scaler.transform(test_data[cols_normalize]),
        columns=cols_normalize,
        index=test_data.index,
    )
    test_join_df = test_data[test_data.columns.difference(cols_normalize)].join(
        norm_test_df
    )
    test_df = test_join_df.reindex(columns=test_data.columns)
    test_df = test_df.reset_index(drop=True)
    return train_df, test_df


In [64]:
train_df, test_df = train_test_scaler(train_df, test_df)

train_df

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s14,s15,s16,s17,s18,s19,s20,s21,R_early,cycle_norm
0,1,1,0.459770,0.166667,0.0,0.0,0.183735,0.406802,0.309757,0.0,...,0.199608,0.363986,0.0,0.333333,0.0,0.0,0.713178,0.724662,130,0.000000
1,1,2,0.609195,0.250000,0.0,0.0,0.283133,0.453019,0.352633,0.0,...,0.162813,0.411312,0.0,0.333333,0.0,0.0,0.666667,0.731014,130,0.002770
2,1,3,0.252874,0.750000,0.0,0.0,0.343373,0.369523,0.370527,0.0,...,0.171793,0.357445,0.0,0.166667,0.0,0.0,0.627907,0.621375,130,0.005540
3,1,4,0.540230,0.500000,0.0,0.0,0.343373,0.256159,0.331195,0.0,...,0.174889,0.166603,0.0,0.333333,0.0,0.0,0.573643,0.662386,130,0.008310
4,1,5,0.390805,0.333333,0.0,0.0,0.349398,0.257467,0.404625,0.0,...,0.174734,0.402078,0.0,0.416667,0.0,0.0,0.589147,0.704502,130,0.011080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,0.477011,0.250000,0.0,0.0,0.686747,0.587312,0.782917,0.0,...,0.194344,0.656791,0.0,0.750000,0.0,0.0,0.271318,0.109500,4,0.540166
20627,100,197,0.408046,0.083333,0.0,0.0,0.701807,0.729453,0.866475,0.0,...,0.188668,0.727203,0.0,0.583333,0.0,0.0,0.124031,0.366197,3,0.542936
20628,100,198,0.522989,0.500000,0.0,0.0,0.665663,0.684979,0.775321,0.0,...,0.212148,0.922278,0.0,0.833333,0.0,0.0,0.232558,0.053991,2,0.545706
20629,100,199,0.436782,0.750000,0.0,0.0,0.608434,0.746021,0.747468,0.0,...,0.203065,0.823394,0.0,0.583333,0.0,0.0,0.116279,0.234466,1,0.548476


In [65]:
#These functions reshape the training and test datasets in three dimension

def gen_sequence(id_df, seq_length, seq_cols):

    """
    sequence generator

    Parameters:

        id_df: Engine ID
        seq_length: Sequence length
        seq_cols: Sequence column

    """

    data_matrix = id_df[seq_cols].values
    num_elements = data_matrix.shape[0]

    for start, stop in zip(
        range(0, num_elements - seq_length), range(seq_length, num_elements)
    ):
        yield data_matrix[start:stop, :]


def sequence_gen(train_data, sequence_length, sequence_cols):

    """
    training sequence generator.

    Returns:
        Generated sequence in the form
        [samples, time_steps, features]

    """
    seq_gen = (
        list(
            gen_sequence(
                train_data[train_data["id"] == id], sequence_length, sequence_cols
            )
        )
        for id in train_data["id"].unique()
    )
    # generate sequences and convert to numpy array
    seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
    return seq_array


# function to generate labels
def gen_labels(id_df, seq_length, label):
    data_matrix = id_df[label].values
    num_elements = data_matrix.shape[0]
    return data_matrix[seq_length:num_elements, :]


def label_generator(train_data, sequence_length, sequence_cols):
    """ generate labels """
    label_gen = [
        gen_labels(train_data[train_data["id"] == id], sequence_length, ["R_early"])
        for id in train_data["id"].unique()
    ]
    label_array = np.concatenate(label_gen).astype(np.float32)
    return label_array



In [66]:
#Specify training sample, training label
seq_array = sequence_gen(train_df, sequence_length, sequence_cols)
label_array = label_generator(train_df, sequence_length, ['R_early'])
print('Data Shape: ', seq_array.shape)
print('Label Shape: ', label_array.shape)
seq_array

Data Shape:  (11631, 90, 3)
Label Shape:  (11631, 1)


array([[[0.33333334, 0.4597701 , 0.        ],
        [0.33333334, 0.6091954 , 0.00277008],
        [0.16666667, 0.25287357, 0.00554017],
        ...,
        [0.33333334, 0.6666667 , 0.24099723],
        [0.33333334, 0.545977  , 0.2437673 ],
        [0.25      , 0.59770113, 0.2465374 ]],

       [[0.33333334, 0.6091954 , 0.00277008],
        [0.16666667, 0.25287357, 0.00554017],
        [0.33333334, 0.54022986, 0.00831025],
        ...,
        [0.33333334, 0.545977  , 0.2437673 ],
        [0.25      , 0.59770113, 0.2465374 ],
        [0.5       , 0.6436782 , 0.24930748]],

       [[0.16666667, 0.25287357, 0.00554017],
        [0.33333334, 0.54022986, 0.00831025],
        [0.41666666, 0.3908046 , 0.01108033],
        ...,
        [0.25      , 0.59770113, 0.2465374 ],
        [0.5       , 0.6436782 , 0.24930748],
        [0.33333334, 0.5574713 , 0.25207755]],

       ...,

       [[0.5       , 0.21839081, 0.29639888],
        [0.5       , 0.6666667 , 0.29916897],
        [0.5       , 0

In [67]:
# Similarly, prepare the test data for model evaluation

In [68]:
# prepare test data for model evaluation
def test_data_eval(test_df, sequence_length, sequence_cols):

    """
    This function pick the last sequence for each
    engine in the test data for model evaluation

    """
    seq_array_test_last = [
        test_df[test_df["id"] == id][sequence_cols].values[-sequence_length:]
        for id in test_df["id"].unique()
        if len(test_df[test_df["id"] == id]) >= sequence_length
    ]
    seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)

    y_mask = [
        len(test_df[test_df["id"] == id]) >= sequence_length
        for id in test_df["id"].unique()
    ]
    label_array_test_last = test_df.groupby("id")["R_early"].nth(-1)[y_mask].values
    label_array_test_last = label_array_test_last.reshape(
        label_array_test_last.shape[0], 1
    ).astype(np.float32)
    return seq_array_test_last, label_array_test_last


In [69]:
#Specify test sample, test label
seq_array_test, label_array_test = test_data_eval(test_df, sequence_length, sequence_cols)
print("Data Shape: ", seq_array_test.shape)
print("Label Shape: ", label_array_test.shape)

Data Shape:  (74, 90, 3)
Label Shape:  (74, 1)


In [70]:
#Specify number of timesteps, features, and output
nb_timesteps=seq_array.shape[1]
nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

print("timesteps = ", nb_timesteps, "\nfeatures = ", nb_features, "\noutput_length = ", nb_out)

timesteps =  90 
features =  3 
output_length =  1


In [71]:
#For conventional models, seq_array, will be used for training, and seq_array_test, will be used for testing
#However, for multihead models, reshape the training sample so that each head is used to learn different features in the training sample

In [72]:
training_sample_multihead = [seq_array[:,:,i].reshape((seq_array.shape[0],sequence_length,1)) for i in range(nb_features)]
len(training_sample_multihead)

3

In [73]:
test_sample_multihead = test_data = [seq_array_test[:,:,i].reshape((seq_array_test.shape[0],sequence_length,1)) for i in range(nb_features)]
len(test_sample_multihead[0][73])

90

In [74]:
# these are the evaluation metrics for the model
def k_score(y_true, y_pred):

    """score metric used for model evaluation"""

    d = y_pred - y_true
    return K.sum(K.exp(d[d >= 0] / 10) - 1) + K.sum(K.exp(-1 * d[d < 0] / 13) - 1)


def rmse(y_true, y_pred):

    """rmse metric used for model evaluation"""

    return K.sqrt(K.mean(K.square(y_pred - y_true)))



In [75]:
# define the new loss function

def newLoss(y_true, y_pred) :

    d = y_pred - y_true

    return 0.95 * (K.mean(K.square(y_pred - y_true))) + 0.05 * (K.sum(K.exp(d[d >= 0] / 10) - 1) + K.sum(K.exp(-1 * d[d < 0] / 13) - 1))


#Create attention_based Multihead CNLSTM

In [76]:
wandb.config = {
  "learning_rate": 3e-5,
  "epochs": 30,
  
}

# ... Define a model

In [77]:
"""
Attention_based multihead CNLSTM
"""
#This is my implemetation of the AMCNLSTM. Feel free to experiment with the architecture

# in_layers, out_layers = list(), list()
# for i in range(nb_features):
#   inputs = Input(shape=(sequence_length,1))
#   rnn1 = Conv1D(filters=64,kernel_size=2,strides=1,padding="same")(inputs)
#   lr1= LeakyReLU()(rnn1)
#   bn1= BatchNormalization()(lr1)
#   rnn2 = Conv1D(filters=64,kernel_size=2,strides=1,padding="same")(bn1)
#   lr2= LeakyReLU()(rnn2)
#   bn2= BatchNormalization()(lr2)
#   rnn3 = LSTM(units=50, return_sequences=True)(bn2)
#   lr3= LeakyReLU()(rnn3)
#   bn3= BatchNormalization()(lr3)
#   att = SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL, kernel_regularizer=keras.regularizers.l2(1e-4), bias_regularizer=keras.regularizers.l1(1e-4), attention_regularizer_weight=1e-4,attention_width=15)(bn3)
#   pool1 = MaxPooling1D(pool_size=2)(att)
#   flat = Flatten()(pool1)
#   # store layers
#   in_layers.append(inputs)
#   out_layers.append(flat)
# # merge heads
# merged = concatenate(out_layers)
# # interpretation
# dense1 = Dense(50, activation='relu')(merged)
# outputs = Dense(nb_out)(dense1)
# model = Model(inputs=in_layers, outputs=outputs)
# optimizer = keras.optimizers.adam_v2.Adam(lr=3e-5)

# model.compile(loss=keras.losses.Huber(),
#               optimizer=optimizer,
#               metrics=[rmse, k_score])

# AMCNLSTM = model
# pydot_ng.find_graphviz()
# plot_model(AMCNLSTM, to_file='AMCNLSTM-90.png', show_shapes=True, rankdir="TB")

"""
AMBiLSTM model (testing)
"""

# in_layers, out_layers = list(), list()
# for i in range(nb_features):
#   inputs = Input(shape=(sequence_length, 1))
#   rnn1 = Bidirectional(keras.layers.LSTM(units=50, return_sequences=True),merge_mode = 'concat')(inputs)
#   lr1= LeakyReLU()(rnn1)
#   bn1= BatchNormalization()(lr1)
#   # att = SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL, kernel_regularizer=keras.regularizers.l2(1e-4), bias_regularizer=keras.regularizers.l1(1e-4), attention_regularizer_weight=1e-4,attention_width=15)(bn1)
#   pool1 = MaxPooling1D(pool_size=2)(bn1)
#   flat = Flatten()(pool1)
#   # store layers
#   in_layers.append(inputs)
#   out_layers.append(flat)
# # merge heads
# merged = concatenate(out_layers)
# # interpretation
# dense1 = Dense(50, activation='relu')(merged)
# outputs = Dense(nb_out)(dense1)
# model = Model(inputs=in_layers, outputs=outputs)
# optimizer = keras.optimizers.adam_v2.Adam(lr=3e-5)

# # model.compile(loss='mse',
# #               optimizer=optimizer,
# #               metrics=[rmse, k_score])
# model.compile(loss=keras.losses.Huber(),
#               optimizer=optimizer,
#               metrics=[rmse, k_score])

# AMBiLSTM = model
# # ABiLSTM.summary()
# # pydot_ng.find_graphviz()
# # plot_model(AMBiLSTM, to_file='MBiLSTM.png', show_shapes=True, rankdir="TB")

"""
AMCNGRU(tested not good)
"""
# from keras.layers.recurrent import GRU
# in_layers, out_layers = list(), list()
# for i in range(nb_features):
#   inputs = Input(shape=(sequence_length,1))
#   rnn3 = Bidirectional(GRU(units=50, recurrent_activation = 'sigmoid', reset_after=True, return_sequences=True))(inputs)
#   lr3= LeakyReLU()(rnn3)
#   bn3= BatchNormalization()(lr3)
#   att = SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL, kernel_regularizer=keras.regularizers.l2(1e-4), bias_regularizer=keras.regularizers.l1(1e-4), attention_regularizer_weight=1e-4,attention_width=15)(bn3)
#   pool1 = MaxPooling1D(pool_size=2)(att)
#   flat = Flatten()(pool1)
#   # store layers
#   in_layers.append(inputs)
#   out_layers.append(flat)
# # merge heads
# merged = concatenate(out_layers)
# # interpretation
# dense1 = Dense(50, activation='relu')(merged)
# outputs = Dense(nb_out)(dense1)
# model = Model(inputs=in_layers, outputs=outputs)
# optimizer = keras.optimizers.adam_v2.Adam(lr=3e-5)

# model.compile(loss=keras.losses.Huber(),
#               optimizer=optimizer,
#               metrics=[rmse, k_score])

# AMBiGRU = model
# pydot_ng.find_graphviz()
# plot_model(AMBiGRU, to_file='AMBiGRU.png', show_shapes=True, rankdir="TB")



'\nAMCNGRU(tested not good)\n'

In [78]:
"""
MFNN model (testing)
"""
in_layers, out_layers = list(), list()
for i in range(nb_features):
  inputs = Input(shape=(sequence_length, 1))
  den1 = Dense(units=50, activation='relu')(inputs)
  lr1= LeakyReLU()(den1)
  bn1= BatchNormalization()(lr1)
  den2 = Dense(units=50, activation='relu')(bn1)
  lr2= LeakyReLU()(den2)
  bn2= BatchNormalization()(lr2)
  den3 = Dense(units=50, activation='relu')(bn2)
  lr3= LeakyReLU()(den3)
  bn3= BatchNormalization()(lr3)
  den4 = Dense(units=50, activation='relu')(bn3)
  lr4= LeakyReLU()(den4)
  bn4= BatchNormalization()(lr4)
  # att = SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL, kernel_regularizer=keras.regularizers.l2(1e-4), bias_regularizer=keras.regularizers.l1(1e-4), attention_regularizer_weight=1e-4,attention_width=15)(bn4)
  pool1 = MaxPooling1D(pool_size=2)(bn4)
  flat = Flatten()(pool1)
  drop = Dropout(0.2)(flat)
  # store layers
  in_layers.append(inputs)
  out_layers.append(drop)
# merge heads
merged = concatenate(out_layers)
# interpretation
dense1 = Dense(50, activation='relu')(merged)
outputs = Dense(nb_out)(dense1)
model = Model(inputs=in_layers, outputs=outputs)
optimizer = keras.optimizers.adam_v2.Adam(lr=3e-5)
model.compile(loss = keras.losses.Huber(),
              optimizer = optimizer,
              metrics = [rmse, k_score])
# model.compile(loss = newLoss,
#               optimizer = optimizer,
#               metrics = [rmse, k_score])

MFNN4 = model
model.summary()

pydot_ng.find_graphviz()
plot_model(MFNN4, to_file='MMLPs.png', show_shapes=True, rankdir="TB")


"""
M2LSTM
"""

# in_layers, out_layers = list(), list()
# for i in range(nb_features):
#   inputs = Input(shape=(sequence_length, 1))
#   rnn1 = LSTM(units=64, return_sequences=True)(inputs)
# #   rnn3 = Bidirectional(keras.layers.LSTM(units=50, return_sequences=True),merge_mode = 'concat')(inputs)
#   lr1= LeakyReLU()(rnn1)
#   bn1= BatchNormalization()(lr1)
#   pool1 = MaxPooling1D(pool_size=2)(bn1)
#   # flat = Flatten()(pool1)
#   # store layers
#   in_layers.append(inputs)
#   out_layers.append(pool1)
# # merge heads
# merged = concatenate(out_layers)
# # interpretation(add attention)

# att = SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL, kernel_regularizer=keras.regularizers.l2(1e-4), bias_regularizer=keras.regularizers.l1(1e-4), attention_regularizer_weight=1e-4,attention_width=15)(merged)
# rnn2 = LSTM(units=64, return_sequences=True)(att)
# dp = Dropout(0.2)(rnn2)
# flat = Flatten()(dp)
# outputs = Dense(nb_out)(flat)
# model = Model(inputs=in_layers, outputs=outputs)
# optimizer = keras.optimizers.adam_v2.Adam(lr=3e-5)
# model.compile(loss = keras.losses.Huber(),
#               optimizer = optimizer,
#               metrics = [rmse, k_score])
# # model.compile(loss=newLoss,
# #               optimizer=optimizer,
# #               metrics=[rmse, k_score])

# MLSTMs = model
# # ABiLSTM.summary()
# pydot_ng.find_graphviz()
# plot_model(MLSTMs, to_file='M2LSTM.png', show_shapes=True, rankdir="TB")



/Users/chengyingzhu/miniforge3/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           [(None, 90, 1)]      0                                            
__________________________________________________________________________________________________
input_19 (InputLayer)           [(None, 90, 1)]      0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           [(None, 90, 1)]      0                                            
__________________________________________________________________________________________________
dense_70 (Dense)                (None, 90, 50)       100         input_18[0][0]                   
____________________________________________________________________________________________

'\nM2LSTM\n'

In [79]:
# Creat model checkpoint to save the best model performance
model_checkpoint = keras.callbacks.ModelCheckpoint(
    filepath="content/sample_data/" + filename + ".h5", save_best_only=True, verbose=1
)

# specify ealy_callback to stop training model if there's no improvement after 5 epochs
early_stopping = keras.callbacks.EarlyStopping(patience=5)

tensorboard = TensorBoard(log_dir='/output/Graph', histogram_freq=0, write_graph=True, write_images=True)

2022-06-12 00:46:26.217341: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-06-12 00:46:26.217353: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-06-12 00:46:26.217457: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


In [80]:
# Plot the model
#It can be seen here that the number of heads= number of features (columns) in the training sample
# pydot.find_graphviz()
# keras.utils.plot_model(model, show_shapes=True, to_file='AMCNLSTM.png',dpi=40)

In [81]:
#Train the multihead model
with tf.device('/cpu:0'):
        train_history=model.fit(training_sample_multihead, label_array, batch_size=32, epochs=30,
        validation_split=0.2, callbacks=[model_checkpoint,WandbCallback()])


2022-06-12 00:46:27.670805: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-06-12 00:46:27.671636: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/30


2022-06-12 00:46:28.468131: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


290/291 [============================>.] - ETA: 0s - loss: 29.4888 - rmse: 38.4100 - k_score: 10227.8369

2022-06-12 00:46:37.697591: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


291/291 [==============================] - 11s 32ms/step - loss: 29.4677 - rmse: 38.3687 - k_score: 10193.8623 - val_loss: 69.5468 - val_rmse: 72.9566 - val_k_score: 163530.0000

Epoch 00001: val_loss improved from inf to 69.54684, saving model to content/sample_data/FD001.h5
Epoch 2/30
291/291 [==============================] - 11s 37ms/step - loss: 19.0497 - rmse: 25.7082 - k_score: 721.5134 - val_loss: 33.5041 - val_rmse: 35.0025 - val_k_score: 3123.6331

Epoch 00002: val_loss improved from 69.54684 to 33.50411, saving model to content/sample_data/FD001.h5
Epoch 3/30
 39/291 [===>..........................] - ETA: 7s - loss: 17.7062 - rmse: 23.7896 - k_score: 502.2395

KeyboardInterrupt: 

wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.


In [ ]:
# load the best checkpoint and evaluate on test set

# This function loads the best model
def load_model_from_disk(filename):

    """Function to load the best model after training"""

    return load_model(
        "content/sample_data/" + filename + ".h5",
        # custom_objects={"rmse": rmse, "k_score": k_score,"SeqSelfAttention":SeqSelfAttention, "newLoss": newLoss})
        custom_objects={"rmse": rmse, "k_score": k_score,"SeqSelfAttention":SeqSelfAttention})

In [ ]:
trained_model = load_model_from_disk(filename)

In [ ]:
#Evaluate the model on test data
with tf.device('/cpu:0'):
    model_evaluation_res = trained_model.evaluate(
        test_sample_multihead, label_array_test, return_dict=True)

In [ ]:
# Predict the degradation trend in test data
y_pred = trained_model.predict(test_data)
#y_pred=np.squeeze(y_pred)
y_pred

In [ ]:
#plot the prediction vs the ground truth
def plot_result(y_true, y_pred, title=None):

    """ A function to plot the predicted RUL vs the target RUL for all engines"""

    fig = plt.figure(figsize = (10.5, 9.5))
    plt.grid(True,lw=0.1)
    plt.rcParams['axes.linewidth'] = 0.4  # set the value globally
    plt.plot(y_true, 'r', linewidth=1,linestyle='--')
    plt.plot(y_pred, 'k', linewidth=1)
    plt.title(title,fontsize=8)
    plt.xlabel('Engine', fontsize=9)
    plt.xticks(fontsize=5)
    plt.ylabel('RUL', fontsize=7)
    plt.yticks(fontsize=5)
    plt.legend(['Actual RUL', 'Predicted RUL'],  loc='upper right', fontsize=5)
    plt.show()
    fig.savefig("content/sample_data/"  + title + '.png', dpi=256, format='png', bbox_inches='tight')
    return fig


In [ ]:
# View the results
plot_result(
    label_array_test, y_pred, title="Model MFNN prediction on " + filename + " test set"
)

print ("sequence length = ", sequence_length ,"\nRMSE = ", K.sqrt(K.mean(K.square(label_array_test - y_pred))))
from sklearn.metrics import r2_score
print ('R2-score = ', round(r2_score(label_array_test,y_pred), 4))
# print ("Adjust_R2 = ", 1-((1-r2_score(label_array_test,y_pred)) * (seq_array_test.shape[0]-1))/(seq_array_test.shape[0]-seq_array_test.shape[2]-1))


per_real_loss=(label_array_test-y_pred)/label_array_test
avg_per_real_loss=sum(abs(per_real_loss))/len(per_real_loss)
print("avg_per_real_loss = ", avg_per_real_loss)

def comput_acc(real,predict,level):
    num_error=0
    for i in range(len(real)):
        if abs(real[i]-predict[i])/real[i]>level:
            num_error+=1
    return 1-num_error/len(real)
print ("不同置信区间内的准确率：", "0.2:",comput_acc(label_array_test,y_pred,0.2),
"0.15:", comput_acc(label_array_test,y_pred,0.15),"0.1:", comput_acc(label_array_test,y_pred,0.1))

In [ ]:

# Test the model on randomly selected engine from the test dataset

# if filename == "FD001":
#     eng_no = [5,21,31,64]
# elif filename =="FD002":
#     eng_no= [11,31]
# elif filename=="FD003":
#     eng_no=[4,10,34,71,96]
# else:
#     eng_no= [1,37]   

# for numb in eng_no:
eng_df = pd.read_csv(
    "random_test/random_eng_FD001-21.csv"
)
seq_array_eng = sequence_gen(eng_df, sequence_length, sequence_cols)
seq_array_eng2 = [seq_array_eng[:,:,i].reshape((seq_array_eng.shape[0],sequence_length,1)) for i in range(nb_features)]
print(seq_array_eng.shape)
label_array_eng = label_generator(eng_df, sequence_length, ["R_early"])
print(label_array_eng.shape)
y_pred_eng = trained_model.predict(seq_array_eng2)
#y_pred_eng = best_model.predict(seq_array_eng)
y_pred_eng=np.squeeze(y_pred_eng)
plot_result(
    label_array_eng,
    y_pred_eng,
    title="Test engine unit 5" + filename
)